In [2]:
pip install prefect

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached cryptography-43.0.1-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
  Using cached ruamel.yaml-0.18.6-py3-none-any.whl.metadata (23 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached h2-4.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached ruamel.yaml.clib-0.2.8-cp312-cp312-win_amd64.whl.metadata (2.3 kB)
  Using cached hyperframe-6.0.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached hpack-4.0.0-py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
   -------- ------------------------------- 1.3/6.0 MB 6.7 MB/s eta 0:00:01
   ------------------- -------------------- 2.9/6.0 MB 7.6 MB/s eta 0:00:01
   ----------------------------- ---------- 4.5/6.0 MB 7.7 MB/s eta 0:00:01
   -------------------------------------- - 5.8/6.0 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 6.0/6.0 MB 7.0 MB/s eta 0:00:00
   ----------

In [22]:

!git clone https://github.com/jonruida/PowerBI_Scraper-webscraping



Cloning into 'PowerBI_Scraper-webscraping'...


In [14]:
pip install prefect_aws

   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   ----------- ---------------------------- 3.7/12.5 MB 18.1 MB/s eta 0:00:01
   ------------------------- -------------- 7.9/12.5 MB 18.7 MB/s eta 0:00:01
   -------------------------------------- - 12.1/12.5 MB 18.9 MB/s eta 0:00:01
   ---------------------------------------- 12.5/12.5 MB 17.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
os.chdir("C:/Users/jon.ruizcarrillo/Desktop/Power_BI_Scraper/Power_BI_API/PowerBI_Scraper-webscraping/")


In [6]:
pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.5 MB 8.4 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/11.5 MB 7.7 MB/s eta 0:00:02
   ---------------- ----------------------- 4.7/11.5 MB 7.7 MB/s eta 0:00:01
   --------------------- ------------------ 6.3/11.5 MB 7.7 MB/s eta 0:00:01
   --------------------------- ------------ 7.9/11.5 MB 7.7 MB/s eta 0:00:01
   -------------------------------- ------- 9.4/11.5 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.6 MB 7.6 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/12.6 MB 8.0 MB/s eta 0:00:02
   --------------- ------------------------ 4.7/12.6 MB 7.7 MB/s eta 0:00:02
   --------

In [13]:
####################
# Author : Kulwinder K.
# Date : 12/12/2023
# This script scrapes the Power BI dashboard from the below website
# https://www.un.org/techenvoy/global-digital-compact
# and gives back the CSV file
# It calls json_to_csv.py file inside the script.
####################
# Co-Author : Emmanuel Letremble
# Date : 08/01/2024
# Improved the script to run in less than 1 min instead of about 1H
#####################

"""
ETL Script for Scraping UN Power BI Dashboard

This script defines the Prefect flow for scraping data from the UN Power BI dashboard.

Functions:
- page_1_scraping: Task to scrape data from page 1 of the Power BI dashboard.
- page_2_scraping: Task to scrape data from page 2 of the Power BI dashboard and merge it with existing data.

Flow:
- omdena_ungdc_etl_scrap_pbi_parent: Prefect flow for scraping data from the UN Power BI dashboard.
  - Scrapes data from page 1.
  - Sets record IDs for page 2.
  - Scrapes data from page 2 and merges it with existing data.

Parameters:
None

Returns:
None
"""

import requests
from pathlib import Path
from typing import Dict, Any

from prefect import flow, task
from prefect_aws import S3Bucket

import pandas as pd
from json_to_csv import extract

In [51]:
# payload for tables and second page sections
payload = {
    "version": "1.0.0",
    "queries": [
        {
            "Query": {
                "Commands": [
                    {
                        "SemanticQueryDataShapeCommand": {
                            "Query": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "d",
                                        "Entity": "Demographics",
                                        "Type": 0,
                                    },
                                    {
                                        "Name": "a",
                                        "Entity": "All Areas combined",
                                        "Type": 0,
                                    },
                                ],
                                "Select": [
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {"Source": "d"}
                                            },
                                            "Property": "Type",
                                        },
                                        "Name": "Demographics.Type",
                                    },
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {"Source": "d"}
                                            },
                                            "Property": "Location formatted",
                                        },
                                        "Name": "Demographics.Location formatted",
                                    },
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {"Source": "d"}
                                            },
                                            "Property": "Contact Name",
                                        },
                                        "Name": "Demographics.Contact Name",
                                    },
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {"Source": "d"}
                                            },
                                            "Property": "Entity Name",
                                        },
                                        "Name": "Demographics.Entity Name",
                                    },
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {"Source": "a"}
                                            },
                                            "Property": "Record ID",
                                        },
                                        "Name": "All Areas combined.Record ID",
                                        "NativeReferenceName": "Record ID1",
                                    },
                                ],
                                "Where": [
                                    {
                                        "Condition": {
                                            "In": {
                                                "Expressions": [
                                                    {
                                                        "Column": {
                                                            "Expression": {
                                                                "SourceRef": {
                                                                    "Source": "a"
                                                                }
                                                            },
                                                            "Property": "Area",
                                                        }
                                                    }
                                                ],
                                                "Values": [
                                                    [
                                                        {
                                                            "Literal": {
                                                                "Value": "'Connect all People'"
                                                            }
                                                        }
                                                    ],
                                                    [
                                                        {
                                                            "Literal": {
                                                                "Value": "'Accountability for Discrimination/Misleading Content'"
                                                            }
                                                        }
                                                    ],
                                                    [
                                                        {
                                                            "Literal": {
                                                                "Value": "'Digital Commons'"
                                                            }
                                                        }
                                                    ],
                                                    [
                                                        {
                                                            "Literal": {
                                                                "Value": "'Regulation of AI'"
                                                            }
                                                        }
                                                    ],
                                                    [
                                                        {
                                                            "Literal": {
                                                                "Value": "'Protect Data'"
                                                            }
                                                        }
                                                    ],
                                                    [
                                                        {
                                                            "Literal": {
                                                                "Value": "'Internet Fragmentation'"
                                                            }
                                                        }
                                                    ],
                                                    [
                                                        {
                                                            "Literal": {
                                                                "Value": "'Human Rights Online'"
                                                            }
                                                        }
                                                    ],
                                                    [
                                                        {
                                                            "Literal": {
                                                                "Value": "'Other Area'"
                                                            }
                                                        }
                                                    ],
                                                ],
                                            }
                                        }
                                    }
                                ],
                                "OrderBy": [
                                    {
                                        "Direction": 2,
                                        "Expression": {
                                            "Aggregation": {
                                                "Expression": {
                                                    "Column": {
                                                        "Expression": {
                                                            "SourceRef": {
                                                                "Source": "a"
                                                            }
                                                        },
                                                        "Property": "Core Principles",
                                                    }
                                                },
                                                "Function": 2,
                                            }
                                        },
                                    }
                                ],
                            },
                            "Binding": {
                                "Primary": {
                                    "Groupings": [{"Projections": [0, 1, 2, 3, 4]}]
                                },
                                "DataReduction": {
                                    "DataVolume": 3,
                                    "Primary": {"Window": {"Count": 500}},
                                },
                                "Version": 1,
                            },
                            "ExecutionMetricsKind": 1,
                        }
                    }
                ]
            },
            "CacheKey": "",
            "QueryId": "",
            "ApplicationContext": {
                "DatasetId": "fb1276c8-e98e-4b26-a7d3-24a40ae84000",
                "Sources": [
                    {
                        "ReportId": "a70c1024-15f3-4f74-aa2e-2f1897acfdb6",
                        "VisualId": "fc23d492ddbe07d7e1a4",
                    }
                ],
            },
        }
    ],
    "cancelQueries": [],
    "modelId": 933989,
}


In [37]:
import requests

def generate_payload(tables, model_id):
    from_clause = []
    select_clause = []

    for i, table in enumerate(tables):
        table_name = f"t{i+1}"
        from_clause.append({
            "Name": table_name,
            "Entity": table["name"],
            "Type": 0
        })
        for column in table["columns"]:
            select_clause.append({
                "Column": {
                    "Expression": {
                        "SourceRef": {
                            "Source": table_name
                        }
                    },
                    "Property": column
                },
                "Name": column
            })

    payload = {
        "version": "1.0.0",
        "modelId": model_id,
        "queries": [
            {
                "Query": {
                    "Commands": [
                        {
                            "SemanticQueryDataShapeCommand": {
                                "Query": {
                                    "Version": 2,
                                    "From": from_clause,
                                    "Select": select_clause
                                }
                            }
                        }
                    ]
                }
            }
        ]
    }

    return payload

# Ejemplo de uso
tables = [
    {
        "name": "Demographics",
        "columns": ["Age", "Region"]
    },
    {
        "name": "Sales",
        "columns": ["TotalSales", "Date"]
    }
]

model_id = 933989  # Reemplaza esto con el ID de tu modelo

api_url = "https://wabi-north-europe-j-primary-api.analysis.windows.net/public/reports/querydata?synchronous=true"
headers = {
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "X-Powerbi-Resourcekey": "84db278f-178b-4a18-a0db-3e57e8113b1f",
}

payload = generate_payload(tables, model_id)
table_data = requests.post(api_url, json=payload, headers=headers).json()

print(table_data)


{'jobIds': ['af83432f-162c-429f-b87c-e08508b81a5a'], 'results': [{'jobId': 'af83432f-162c-429f-b87c-e08508b81a5a', 'result': {'data': {'timestamp': '2024-09-09T11:06:35.637Z', 'rootActivityId': '3c461ede-8e13-42fb-8c1e-ca519c4f30c9', 'fromCache': False, 'dsr': {'DataShapes': [{'odata.error': {'code': 'CouldNotResolveSemanticQueryDefinition', 'source': 'User', 'message': {'lang': 'en-US', 'value': "Could not resolve QueryDefinition due to invalid SourceEntity reference 'Sales'. This error often happens after the Entity was renamed or deleted. Please update the query/visual to use the new name or undo the name change."}, 'azure:values': [{'timestamp': '2024-09-09T11:06:35.7312296Z'}, {'additionalMessages': [{'Code': 'CouldNotResolveSemanticQueryDefinition', 'Severity': 'Error', 'Message': "Could not resolve QueryDefinition due to invalid SourceEntity reference 'Sales'. This error often happens after the Entity was renamed or deleted. Please update the query/visual to use the new name or 

In [54]:
api_url = "https://wabi-north-europe-j-primary-api.analysis.windows.net/public/reports/querydata?synchronous=true"
headers = {
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "X-Powerbi-Resourcekey": "84db278f-178b-4a18-a0db-3e57e8113b1f",
}

table_data = requests.post(api_url, json=payload, headers=headers).json()


JSONDecodeError: Expecting value: line 2 column 1 (char 2)

In [58]:
import requests
import time

# https://app.powerbi.com/groups/reports//?experience=power-bi&bookmarkGuid=Bookmarkb48a25697bac0cdc4290
# Configuración
group_id = '/e972c2cb-848c-4d27-b3b0-dcd2fc170e81'  # Reemplaza con tu groupId
report_id = '82ef2309-602f-48d7-af7e-712cd9d464aa'  # Reemplaza con tu reportId
access_token = 'YOUR_ACCESS_TOKEN'  # Reemplaza con tu token de acceso

# URL y headers para la solicitud de exportación
api_url_export = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/reports/{report_id}/ExportTo"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Cuerpo de la solicitud para exportar a PDF
payload = {
    "format": "PDF",
    "powerBIReportConfiguration": {
        "pages": [
            {
                "pageName": "ReportSectiond56ca7ee2eb409a0ffad"  # Reemplaza con el nombre de la sección de tu informe
            }
        ]
    }
}

# Realizar la solicitud de exportación
response = requests.post(api_url_export, json=payload, headers=headers)
export_job = response.json()

# Obtener el ID del trabajo de exportación
job_id = export_job['id']

# Verificar el estado del trabajo de exportación
api_url_status = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/reports/{report_id}/exports/{job_id}"
while True:
    status_response = requests.get(api_url_status, headers=headers)
    status = status_response.json()
    if status['status'] == 'Succeeded':
        file_url = status['resourceLocation']
        break
    elif status['status'] == 'Failed':
        raise Exception("Export failed")
    time.sleep(5)  # Esperar 5 segundos antes de verificar nuevamente

# Descargar el archivo exportado
file_response = requests.get(file_url, headers=headers)
with open("exported_report.pdf", "wb") as file:
    file.write(file_response.content)

print("Exportación completada y archivo descargado.")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [53]:
table_data

{'jobIds': ['f17e723a-9c0a-4d77-9d41-dcbc7b304466'],
 'results': [{'jobId': 'f17e723a-9c0a-4d77-9d41-dcbc7b304466',
   'result': {'data': {'timestamp': '2024-09-09T13:25:17.777Z',
     'rootActivityId': '1a418d41-e796-4dd4-a865-983c673f50c0',
     'descriptor': {'Select': [{'Kind': 1,
        'Depth': 0,
        'Value': 'G0',
        'GroupKeys': [{'Source': {'Entity': 'Demographics',
           'Property': 'Type'},
          'Calc': 'G0',
          'IsSameAsSelect': True}],
        'Name': 'Demographics.Type'},
       {'Kind': 1,
        'Depth': 0,
        'Value': 'G1',
        'GroupKeys': [{'Source': {'Entity': 'Demographics',
           'Property': 'Location formatted'},
          'Calc': 'G1',
          'IsSameAsSelect': True}],
        'Name': 'Demographics.Location formatted'},
       {'Kind': 1,
        'Depth': 0,
        'Value': 'G2',
        'GroupKeys': [{'Source': {'Entity': 'Demographics',
           'Property': 'Contact Name'},
          'Calc': 'G2',
          'IsSa

In [50]:
import requests

def get_model_structure(api_url, headers, payload):
    response = requests.post(api_url, json=payload, headers=headers)
    response.raise_for_status()  # Lanza una excepción si la respuesta contiene un error HTTP
    return response.json()

# URL y headers para la consulta inicial
api_url_structure = "https://wabi-north-europe-j-primary-api.analysis.windows.net/public/reports/querydata?synchronous=true"
headers = {
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "X-Powerbi-Resourcekey": "84db278f-178b-4a18-a0db-3e57e8113b1f",
}

# Payload para obtener la estructura del modelo
payload_structure = {
    "version": "1.0.0",
    "queries": [
        {
            "Query": {
                "Commands": [
                    {
                        "SemanticQueryDataShapeCommand": {
                            "Query": {
                                "Version": 2,
                                "From": [],
                                "Select": []
                            }
                        }
                    }
                ]
            }
        }
    ],    "cancelQueries": [],
    "modelId": 933989,

}

try:
    model_structure = get_model_structure(api_url_structure, headers, payload_structure)
    print(model_structure)
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except requests.exceptions.RequestException as req_err:
    print(f"Request error occurred: {req_err}")
except ValueError as json_err:
    print(f"JSON decode error: {json_err}")


{'jobIds': ['4f6328fa-a9df-4fc8-82d8-72fb7091d2cd'], 'results': [{'jobId': '4f6328fa-a9df-4fc8-82d8-72fb7091d2cd', 'result': {'data': {'timestamp': '2024-09-09T11:19:54.89Z', 'rootActivityId': 'c2c30c68-0afe-46bb-ad89-b5ebd2cd44f0', 'fromCache': False, 'dsr': {'DataShapes': [{'odata.error': {'code': 'InvalidOrMalformedSemanticQueryDefinition', 'source': 'PowerBI', 'message': {'lang': 'en-US', 'value': 'The QueryDefinition Select is missing or empty. A semantic query must contain at least one Select'}, 'azure:values': [{'timestamp': '2024-09-09T11:19:54.890813Z'}, {'additionalMessages': [{'Code': 'InvalidOrMalformedSemanticQueryDefinition', 'Severity': 'Error', 'Message': 'The QueryDefinition Select is missing or empty. A semantic query must contain at least one Select'}]}]}}]}}}}]}


In [ ]:
def page_2_scraping(
    api_url: str, payload_p2: Dict[str, Any], headers: Dict[str, str], df: pd.DataFrame
) -> pd.DataFrame:
    """
    Task to scrape data from page 2 of the Power BI dashboard and merge it with existing data.

    Parameters:
    - api_url (str): The API URL for the Power BI dashboard.
    - payload_p2 (Dict[str, Any]): The JSON payload for the API request for page 2.
    - headers (Dict[str, str]): The headers for the API request.
    - df (pd.DataFrame): The existing DataFrame.

    Returns:
    - pd.DataFrame: The DataFrame with additional columns scraped from page 2.
    """

    # SET TOPICS
    # Topic names are manually added from the Page 2 of Power BI
    # It would be better if this is also automated to make it more agile.
    topics = [
        "Accountability for Discrimination/Misleading Content",
        "Connect all People",
        "Digital Commons",
        "Human Rights Online",
        "Internet Fragmentation",
        "Other Area",
        "Protect Data",
        "Regulation of AI",
    ]

    # POPULATE TOPICS
    for topic in topics:
        print(">>> TOPIC:", topic)
        clean_topic = topic.replace(" ", "_").replace("/", "_")

        payload_p2["queries"][0]["Query"]["Commands"][0][
            "SemanticQueryDataShapeCommand"
        ]["Query"]["Where"][0]["Condition"]["In"]["Values"][0][0]["Literal"][
            "Value"
        ] = f"'{topic}'"

        result = requests.post(api_url, json=payload_p2, headers=headers).json()
        query_df = extract(result)

        # Merge the `Core Principles` and `Commitments, pledges or actions` columns with the existing DF
        cols = ["Record ID", "Core Principles", "Commitments, pledges or actions"]
        tmp_df = query_df.loc[:, cols]
        tmp_df.rename(
            columns={
                "Core Principles": f"Core_Principle__{clean_topic}",
                "Commitments, pledges or actions": f"Commitments_pledges_or_actions__{clean_topic}",
            },
            inplace=True,
        )

        left = df.set_index("Record ID", drop=False)
        right = tmp_df.set_index("Record ID", drop=True)
        df = left.join(right, how="left", lsuffix="", rsuffix=f"__{clean_topic}")

    else:
        print(">>> PROCESS DESCRIPTION")
        # Merge the `Process description` column on the last call
        left = df.set_index("Record ID", drop=False)

        tmp_df = query_df.loc[:, ["Record ID", "Process description"]]
        right = tmp_df.set_index("Record ID", drop=True)

        df = left.join(right, how="left", lsuffix="", rsuffix=f"")

    return df


def omdena_ungdc_etl_scrap_pbi_parent() -> None:
    """
    Prefect flow for scraping data from the UN Power BI dashboard.

    Returns:
    None
    """

    # payload for tables and second page sections
    payload_p1 = {
        "version": "1.0.0",
        "queries": [
            {
                "Query": {
                    "Commands": [
                        {
                            "SemanticQueryDataShapeCommand": {
                                "Query": {
                                    "Version": 2,
                                    "From": [
                                        {
                                            "Name": "d",
                                            "Entity": "Demographics",
                                            "Type": 0,
                                        },
                                        {
                                            "Name": "a",
                                            "Entity": "All Areas combined",
                                            "Type": 0,
                                        },
                                    ],
                                    "Select": [
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "d"}
                                                },
                                                "Property": "Type",
                                            },
                                            "Name": "Demographics.Type",
                                        },
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "d"}
                                                },
                                                "Property": "Location formatted",
                                            },
                                            "Name": "Demographics.Location formatted",
                                        },
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "d"}
                                                },
                                                "Property": "Contact Name",
                                            },
                                            "Name": "Demographics.Contact Name",
                                        },
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "d"}
                                                },
                                                "Property": "Entity Name",
                                            },
                                            "Name": "Demographics.Entity Name",
                                        },
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "a"}
                                                },
                                                "Property": "Record ID",
                                            },
                                            "Name": "All Areas combined.Record ID",
                                            "NativeReferenceName": "Record ID1",
                                        },
                                    ],
                                    "Where": [
                                        {
                                            "Condition": {
                                                "In": {
                                                    "Expressions": [
                                                        {
                                                            "Column": {
                                                                "Expression": {
                                                                    "SourceRef": {
                                                                        "Source": "a"
                                                                    }
                                                                },
                                                                "Property": "Area",
                                                            }
                                                        }
                                                    ],
                                                    "Values": [
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Connect all People'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Accountability for Discrimination/Misleading Content'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Digital Commons'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Regulation of AI'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Protect Data'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Internet Fragmentation'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Human Rights Online'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Other Area'"
                                                                }
                                                            }
                                                        ],
                                                    ],
                                                }
                                            }
                                        }
                                    ],
                                    "OrderBy": [
                                        {
                                            "Direction": 2,
                                            "Expression": {
                                                "Aggregation": {
                                                    "Expression": {
                                                        "Column": {
                                                            "Expression": {
                                                                "SourceRef": {
                                                                    "Source": "a"
                                                                }
                                                            },
                                                            "Property": "Core Principles",
                                                        }
                                                    },
                                                    "Function": 2,
                                                }
                                            },
                                        }
                                    ],
                                },
                                "Binding": {
                                    "Primary": {
                                        "Groupings": [{"Projections": [0, 1, 2, 3, 4]}]
                                    },
                                    "DataReduction": {
                                        "DataVolume": 3,
                                        "Primary": {"Window": {"Count": 500}},
                                    },
                                    "Version": 1,
                                },
                                "ExecutionMetricsKind": 1,
                            }
                        }
                    ]
                },
                "CacheKey": "",
                "QueryId": "",
                "ApplicationContext": {
                    "DatasetId": "fb1276c8-e98e-4b26-a7d3-24a40ae84000",
                    "Sources": [
                        {
                            "ReportId": "a70c1024-15f3-4f74-aa2e-2f1897acfdb6",
                            "VisualId": "fc23d492ddbe07d7e1a4",
                        }
                    ],
                },
            }
        ],
        "cancelQueries": [],
        "modelId": 933989,
    }

    payload_p2 = {
        "version": "1.0.0",
        "queries": [
            {
                "Query": {
                    "Commands": [
                        {
                            "SemanticQueryDataShapeCommand": {
                                "Query": {
                                    "Version": 2,
                                    "From": [
                                        {
                                            "Name": "a",
                                            "Entity": "All Areas combined",
                                            "Type": 0,
                                        },
                                        {
                                            "Name": "b",
                                            "Entity": "Demographics",
                                            "Type": 0,
                                        },
                                    ],
                                    "Select": [
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "a"}
                                                },
                                                "Property": "Record ID",
                                            },
                                            "Name": "Record ID",
                                        },
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "a"}
                                                },
                                                "Property": "Core Principles",
                                            },
                                            "Name": "All Areas combined.Core Principles",
                                        },
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "a"}
                                                },
                                                "Property": "Commitments, pledges or actions",
                                            },
                                            "Name": "All Areas combined.Commitments, pledges or actions",
                                        },
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "b"}
                                                },
                                                "Property": "Process description",
                                            },
                                            "Name": "Demographics.Process description",
                                        },
                                    ],
                                    "OrderBy": [
                                        {
                                            "Direction": 2,
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {"Source": "a"}
                                                    },
                                                    "Property": "Core Principles",
                                                    # "Property": "Record ID",
                                                    # "Property": "Area",
                                                }
                                            },
                                        }
                                    ],
                                    "Where": [
                                        {
                                            "Condition": {
                                                "In": {
                                                    "Expressions": [
                                                        {
                                                            "Column": {
                                                                "Expression": {
                                                                    "SourceRef": {
                                                                        "Source": "a"
                                                                    }
                                                                },
                                                                "Property": "Area",
                                                            }
                                                        }
                                                    ],
                                                    "Values": [
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'Accountability for Discrimination/Misleading Content'"
                                                                }
                                                            }
                                                        ],
                                                    ],
                                                }
                                            }
                                        },
                                        {
                                            "Condition": {
                                                "In": {
                                                    "Expressions": [
                                                        {
                                                            "Column": {
                                                                "Expression": {
                                                                    "SourceRef": {
                                                                        "Source": "a"
                                                                    }
                                                                },
                                                                "Property": "Record ID",
                                                            }
                                                        }
                                                    ],
                                                    "Values": [
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'638159590404564717'"
                                                                }
                                                            },
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'638183478316757857'"
                                                                }
                                                            },
                                                        ],
                                                    ],
                                                }
                                            }
                                        },
                                    ],
                                },
                                "Binding": {
                                    "Primary": {
                                        "Groupings": [{"Projections": [0, 1, 2, 3]}]
                                    },
                                    # "Primary": {"Groupings": [{"Projections": [0]}]},
                                    "DataReduction": {
                                        "DataVolume": 3,
                                        "Primary": {"Top": {}},
                                        # "Primary": {"Window": {"Count": 500}},
                                    },
                                    "Version": 1,
                                },
                                "ExecutionMetricsKind": 1,
                            }
                        }
                    ]
                },
                "CacheKey": "",
                "QueryId": "",
                "ApplicationContext": {
                    "DatasetId": "fb1276c8-e98e-4b26-a7d3-24a40ae84000",
                    "Sources": [
                        {
                            "ReportId": "a70c1024-15f3-4f74-aa2e-2f1897acfdb6",
                            "VisualId": "58d5b49030dc6e9c802b",
                        }
                    ],
                },
            }
        ],
        "cancelQueries": [],
        "modelId": 933989,
    }

    # api url copied form the Headers section of Network>XHR
    api_url = "https://wabi-north-europe-j-primary-api.analysis.windows.net/public/reports/querydata?synchronous=true"
    headers = {
        "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "X-Powerbi-Resourcekey": "84db278f-178b-4a18-a0db-3e57e8113b1f",
    }

    ##### START SCRAPING P1 #####

    df = page_1_scraping(api_url, payload_p1, headers)

    ##### SET RECORD IDS #####

    record_id = df["Record ID"].tolist()
    ids = [
        [
            {"Literal": {"Value": f"'{x}'"}},
        ]
        for x in record_id
    ]
    payload_p2["queries"][0]["Query"]["Commands"][0]["SemanticQueryDataShapeCommand"][
        "Query"
    ]["Where"][1]["Condition"]["In"]["Values"] = ids

    ##### START SCRAPING P2 #####

    local_dir = "data"
    export_path = Path(local_dir, "powerBI.csv")
    df = page_2_scraping(api_url, payload_p2, headers, df)
    df.to_csv(export_path, index=False)

    print("Scrapping completed")


if __name__ == "__main__":
    omdena_ungdc_etl_scrap_pbi_parent()